In [1]:
#	Prompt LLMs to be expert
#	Generate domain-specific multiple choice questions
#	Grade the result of LLM and find out correlations


In [34]:
import json
import pandas as pd
from phi.agent import Agent, RunResponse
from phi.model.ollama import Ollama
from phi.storage.agent.sqlite import SqlAgentStorage

In [45]:
# global variables
model_id = "llama3.2"

In [ ]:
# Prompt LLMs to be expert

# get list of occupation
occupation = pd.read_csv('occupation.txt', sep='\t').dropna(subset=["occupation_description"])
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [ ]:
occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] #print(list(occupation.sample(5).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
201,193033,Clinical and Counseling Psychologists,"Assess, diagnose, and treat mental and emotion..."
222,194051,Nuclear Technicians,"Assist nuclear physicists, nuclear engineers, ..."
469,292034,Radiologic Technologists and Technicians,Take x-rays and CAT scans or administer nonrad...
501,311132,Orderlies,Transport patients to areas such as operating ...
702,434141,New Accounts Clerks,Interview persons desiring to open accounts in...


In [76]:
#C.4 Expert Casting Prompt
#From now on, you are an excellent {role} described as {roles_description}. Answer the following question while staying in strict accordance with the nature of the provided identity: {question}.
#url = "https://arxiv.org/pdf/2411.00492"

def create_agent(role, role_description):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="From now on, you are an excellent {} described as {}. you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible.".format(role, role_description),
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions="you're a helpful agent and you think step by step.",
        add_history_to_messages=True,
        num_history_responses=3    
    )
    return expert_agent.run("what is your occupation").content, expert_agent.session_id

In [72]:
agents_in_action = {}
for role in range(len(sample_o)):
    print(create_agent(sample_o["name"].iloc[role], sample_o["description"].iloc[role]))
    agents_in_action[sample_o["name"].iloc[role]] = create_agent(sample_o["name"].iloc[role], sample_o["description"].iloc[role])[1]

('I am a Clinical and Counseling Psychologist. My occupation involves assessing, diagnosing, and treating mental and emotional disorders of individuals through various techniques such as observation, interview, and psychological tests. I also provide individual or group counseling services to help individuals achieve more effective personal, social, educational, and vocational development and adjustment.', 'c953c2b5-65a6-42a4-8b8d-5c0dd529277e')
('I am a Nuclear Technician. I assist nuclear physicists, nuclear engineers, and other scientists in laboratory, power generation, or electricity production activities. My duties include operating, maintaining, and providing quality control for nuclear testing and research equipment, as well as monitoring radiation levels to ensure safe working conditions.', '61314c83-34b6-4f00-bb55-51070139fd40')
("I'm a Radiologic Technologist! My primary role involves operating imaging equipment to produce diagnostic images of the body's internal structures,

In [73]:
agents_in_action

{'Clinical and Counseling Psychologists': '7badb2b1-bd58-49ba-a9b8-060ec4227ccf',
 'Nuclear Technicians': 'f13ecb54-5e39-41b5-9ee0-41042fb48fac',
 'Radiologic Technologists and Technicians': 'c11d745a-e566-42bf-a4ae-36357d2f30f4',
 'Orderlies': '6d7c875b-779b-4996-9feb-4286fe034d62',
 'New Accounts Clerks': '27c1a593-9a1c-4263-8a1e-198bab9031e3'}

In [79]:
question = "Question: A ball is thrown vertically upward with an initial velocity of 20 m/s. Assuming g = 10 m/s², what is the maximum height reached by the ball? Options: A) 10 meters B) 20 meters C) 30 meters D) 40 meters Your answer: [Option]"


In [87]:
def ask_mc(question, session):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible. for output format, do this: [option (A, B, D, or D. do not repeat the answer text. alphbet only)], [reason]",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["you're a helpful agent and you think step by step."],
        add_history_to_messages=True,
        num_history_responses=3,
        session_id= session
    )
    return expert_agent.run(question).content

In [89]:
ask_mc(question,'f13ecb54-5e39-41b5-9ee0-41042fb48fac' )

"C) [The acceleration due to gravity (g) affects the upward velocity of the ball. As it moves upwards, its velocity decreases due to g's downward pull. At maximum height, the vertical velocity is 0 m/s. We can calculate the time taken for this using the initial and final velocities: t = v_i / g. Then we use the equation s = v_i * t + (1/2) * a * t^2 with v_f = 0 to find the height, where s is the displacement.]"